In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys

sys.path.insert(0, './denoising-diffusion-pytorch/')

In [ ]:
import torch


torch.__version__

In [ ]:
device = torch.device('mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
device

In [ ]:
import torch
from denoising_diffusion_pytorch import Unet, GaussianDiffusion

model = Unet(
    dim = 64,
    dim_mults = (1, 2, 4, 8)
).to(device)

diffusion = GaussianDiffusion(
    model,
    image_size = 256,
    timesteps = 256,   # number of steps
    loss_type = 'l1'    # L1 or L2
).to(device)

# training_images = torch.rand(8, 3, 128, 128) # images are normalized from 0 to 1
# loss = diffusion(training_images)
# loss.backward()
# # after a lot of training

sampled_images = diffusion.sample(batch_size=2)
sampled_images.shape # (4, 3, 128, 128)

In [ ]:
import matplotlib.pyplot as plt

from torchvision.transforms import ToPILImage


img_t = sampled_images[0]
plt.imshow(img_t.cpu().numpy().transpose(1, 2, 0))
plt.show()

In [ ]:
import torchvision.transforms as T

from PIL import Image
from torch.utils.data import Dataset


class CropImageDataset(Dataset):
    def __init__(self, image, resolution):
        super(Dataset, self).__init__()
        self.image = T.ToTensor()(image)
        _, h, w = self.image.shape
        self.crop_size = int(min(h, w) * 0.95)
        self.resolution = resolution
        self.transforms = T.Compose([
            T.RandomCrop(self.crop_size),
            T.Resize(self.resolution)
        ])
        
    def __len__(self):
        return 10
    
    def __getitem__(self, idx):
        tensor = self.transforms(self.image)
        return tensor
    
    
image = Image.open('pic2.jpeg')
dataset = CropImageDataset(image, resolution=128)
T.ToPILImage()(dataset[0])

In [ ]:
!pip install pytorch_lightning